In [31]:
import advent
advent.scrape(2016, 22)
data = advent.get_lines(22)[2:]
lines = []
for line in data: # [data[0], data[488]]:
    lines.append((
        int(line[16:18].replace('-', '')),
        int(line[19:21].replace('y', '')),
        int(line[23:27]),
        int(line[29:33]),
        int(line[36:40]),

    ))
# x, y, total, used, free

def viable(a, b):
    if a[3] == 0: return False
    if a[0] == b[0] and a[1] == b[1]: return False
    return a[3] <= b[4]

result = 0
for i in range(len(lines)):
    for j in range(len(lines)):
        #print(i, j, viable(lines[i], lines[j]))
        if viable(lines[i], lines[j]):
            result += 1
print(result)


976


In [32]:
from typing import Iterable
from advent.maze import solve_maze
# Well well well.
# Let's try the usual brute force BFS/dijkstra storing the whole board as a node
# The board itself is 37x27 and each position contains 3 numbers: used, avail, payload

start_r = [(line[3], line[4], 0) for line in lines]
start_r[972] = (start_r[972][0], start_r[972][1], 1)
# payload data. 972 i just manually looked up but it's actually just (37-1) * 27

# Now the trick (because I tried it without and it took too long):
# There are only three types of nodes: normal (1, 0, x), empty (0, 1, x) and big (2, 0, x)
start_s = []
for s in start_r:
    # Total capacity is always at least 85 to 94, and data is less than 80 so should always fit
    assert ((s[0] > 100 and s[1] < 50)  or \
            (s[0] == 0 and s[1] > 80) or \
            (80 > s[0] >= 60 and s[1] <= 30 and (85 <= s[0] + s[1] <= 94)))
    if s[0] == 0 and s[1] > 80: start_s.append((0, 1, s[2])) # empty
    elif s[0] > 100 and s[1] < 50: start_s.append((99, 0, s[2])) # big
    elif s[0] >= 60 and s[0] < 80 and s[1] <= 30: start_s.append((1, 0, s[2])) # normal
    else: raise ValueError(s)

Board = tuple[tuple[tuple[int, int, int], ...], ...]
start: Board = tuple([
    tuple(start_s[i:(i+27)])
    for i in range(0, len(lines), 27)
])

assert start[36][0][2] == 1  and start[35][26][2] == 0 and start[36][1][2] == 0 # Assert the payload is there

print(start, len(start), len(start[0]), len(start[36]))


(((1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0)), ((1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0)), ((1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0)), ((1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1, 0, 0), (1

In [33]:
tadd = lambda x, y: (x[0] + y[0], x[1] + y[1])

def viable(a: tuple[int, int, int], b:tuple[int, int, int]):
    return a[0] > 0 and a[0] <= b[1]

assert        viable((1, 0, 0), (0, 1, 0)) and not viable((0, 1, 0), (1, 0, 0)) \
      and not viable((2, 0, 0), (0, 1, 0)) and not viable((0, 1, 0), (2, 0, 0)) \
      and not viable((1, 0, 0), (2, 0, 0)) and not viable((2, 0, 0), (1, 0, 0))

def move(a: tuple[int, int, int], b:tuple[int, int, int]):
    # Assumes its viable, move data from node a to node b
    # This means (1, 0, x), (0, 1, 0) becomes (0, 1, 0), (1, 0, x)
    #return (0, a[0] + a[1], 0), (a[0] + b[0], b[1] - a[0], a[2] + b[2])
    return (0, 1, 0), (1, 0, a[2]) # Only works for the simplified problem

def adj_coords(coord: tuple[int, int]) -> Iterable[tuple[int, int]]:
    offsets = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    for offset in offsets:
        new = tadd(coord, offset)
        if new[0] >= 0 and new[0] < 37 and new[1] >= 0 and new[1] < 27:
            yield new

def adjacent(board: Board) -> Iterable[tuple[Board, int]]:
    for i in range(37):
        for j in range(27):
            for coord in adj_coords((i, j)):
                k, l = coord
                if viable(board[i][j], board[k][l]):
                    a, b = move(board[i][j], board[k][l])
                    newboard = [list(board[m]) for m in range(37)]
                    newboard[i][j] = a
                    newboard[k][l] = b
                    #print(i, j, k, l)
                    yield tuple([tuple(newboard[m]) for m in range(37)]), 1

def is_target(board: Board):
    return board[0][0][2] == 1


In [35]:
# What makes this so hard is that due to the size it's basicaly impossible to check if adjacent is correct since the board is so big

# Edit: seeing the solution below, I suspect that this would have worked, it's just too slow
# r = solve_maze(start, is_target, adjacent)
# print(r[0])

In [36]:
# It didn't work, which I'm fairly sure there is some bug. I had two choices: either go debug the adjacent function, which I didn't want to do
# Or make another board, which simplifies it even more: only track the position of the payload and the empty and bigs
from typing import NamedTuple
from advent.utils import tadd2

class SmartBoard(NamedTuple):
    payload: tuple[int, int]
    empty: tuple[int, int]
    bigs: frozenset[tuple[int, int]]
    size: tuple[int, int] # 37x27

    def in_bound(self, coord: tuple[int, int]):
        return self.size[0] > coord[0] >= 0 and 0 <= coord[1] < self.size[1]
    
    def is_done(self):
        return self.payload == (0, 0)
    
    def move_empty(self):
        offsets = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        for i in offsets:
            o = tadd2(self.empty, i)
            if self.in_bound(o) and o not in self.bigs:
                yield o

def adjacent_smart(board: SmartBoard):
    for new_empty in board.move_empty():
        if new_empty == board.payload:
            # Move the payload
            yield SmartBoard(board.empty, new_empty, board.bigs, board.size), 1
        else:
            yield SmartBoard(board.payload, new_empty, board.bigs, board.size), 1

bigs = []
for line in lines:
    if line[2] > 100:
        assert line[3] > 100 and line[4] < 30
        bigs.append((line[0], line[1]))
start = SmartBoard((36, 0), (20, 6), frozenset(bigs), (37, 27))

_ = solve_maze(start, lambda b: b.is_done(), adjacent_smart)

  0%|          | 0/1 [00:00<?, ?it/s]

Final path length: 209
